In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime
import time
import re

from pymongo import MongoClient

pd.set_option('display.max_columns', 100)

import psycopg2
import pymongo

In [2]:
conn = psycopg2.connect("host=172.32.1.126 port=5432 user=linkshops password=$dpdlvmflf4 dbname=LINKSHOPS-20211121")

curs = conn.cursor()

In [3]:
from_date = '2021-05-01'
to_date = '2021-06-01'

In [122]:

order = {'SQL' : "select \"id\"," + (
                    "\"buyerId\",") + ( 
                    "\"finalTotalKRW\",") + ( 
                    "\"finalSubtotalKRW\",") + ( 
                    "\"finalHandlingFeeKRW\",") + ( 
                    "\"finalShippingCostKRW\",") + (
                    "\"finalTaxKRW\",") + (
                    "\"handlingFeeKRW\",") + (
                    "\"subtotalKRW\",") + (
                    "\"taxKRW\","
                    "\"createdAt\"") + (
                    " from orders WHERE ") + (
                    f"\"createdAt\" > '{from_date}' AND ") + (
                    f"\"createdAt\" < '{to_date}'"),                                    
      'columns' : ['orderId',
                   'buyerId',
                   'finalTotalKRW',
                   'finalSubtotalKRW',
                   'finalHandlingFee',
                   'finalShippingCost',
                   'finalTax',
                   'handlingFeeKRW',
                   'subtotalKRW',
                   'taxKRW',
                     'createdAt']}
fee = {'SQL' : "SELECT \"order_id\"," + (
                    "\"dc_product_amount\",") + ( 
                    "\"dc_handling_fee_amount\",") + (
                    "\"dc_quality_inspection_fee_amount\",") + (
                    "\"dc_order_shipping_cost_amount\",") + (
                    "\"dc_additional_payment_amount\"") + (
                    "FROM tb_order_fee WHERE ") + (
                    f"\"created_at\" > '{from_date}' AND ") + (
                    f"\"created_at\" < '{to_date}'"),   
       'columns' : ['orderId',
                    'dcSubtotal',
                    'dcHandlingFee',
                    'dcQualityInspectionFee',
                    'dcShippingCost',
                    'dcAdditional']}


In [79]:
order_prod = {'SQL' :"select \"id\"," + (
                        "\"orderId\",") + (
                        "\"product\",") + (
                        "\"buyerId\",") + (
                        "\"brand\",") + (
                        "\"quantity\",") + (
                        "\"finalQuantity\",") + (
                        "\"finalKRW\",") + (
                        "\"cmUnclePickupQuantity\",") + (
                        "\"cmInspectionQuantity\",") + (
                        "\"createdAt\"") + (
                        " from order_products WHERE ") + (
                        f"\"createdAt\" > '{from_date}' AND ") + (
                        f"\"createdAt\" < '{to_date}'"),                                    
              'columns' : ['orderProductId',
                           'orderId',
                           'product',
                           'buyerId',
                           'brand',
                           'quantity',
                           'finalQuantity',
                           'finalKRW',
                           'unclepickup',
                           'inspection_quantity',
                            'createdAt']}

In [148]:
prod_fee = {'SQL' : "select \"id\"," + (
                            "\"order_id\",") + (
                            "\"item_id\",") + (
                            "\"unit_product_supply\",") + (
                            "\"unit_handling_fee_amount\",") + (
                            "\"unit_quality_inspection_fee_amount\",") + (
                            "\"unit_product_vat\",") + (
                            "\"dc_unit_product_supply\",") + ( 
                            "\"dc_unit_handling_fee_amount\",") + (
                            "\"dc_unit_quality_inspection_fee_amount\",") + (
                            "\"dc_unit_product_vat\",") + (
                            "\"created_at\"") + (
                            " from tb_order_product_fee WHERE ") + (
                            f"\"created_at\" > '{from_date}' AND ") + (
                            f"\"created_at\" < '{to_date}'"),
            'columns' : ['prodFeeId',
                        'orderId',
                        'orderProductId',
                        'unit_product_supply',
                        'unit_handling_fee_amount', 
                        'unit_quality_inspection_fee_amount', 
                        'unit_product_vat',
                        'dc_unit_product_supply',
                        'dc_unit_handling_fee_amount', 
                        'dc_unit_quality_inspection_fee_amount', 
                        'dc_unit_product_vat',                        
                        'created_at'] }

In [8]:
refund_order = {'SQL' : "select \"id\"," + (
                        "\"order_id\",") + (
                        "\"target_order_product_id\",") + (
                        "\"total_amount\",") + (
                        "\"type\",") + (
                        "\"created_at\"") + (
                        " from tb_refund_order_details WHERE ") + (
                        f"\"created_at\" > '{from_date}' AND ") + (
                        f"\"created_at\" < '{to_date}'"),
                'columns' : ['id',
                            'order_id',
                            'orderProductId',
                            'shipment_amount', 
                            'type',
                            'created_at']
               }

In [9]:
exhibition = {'SQL' : "select * " + (
                            " from tb_exhibition"),                                         
              'columns' : ['id',
                           'name',
                           'path',
                           'description',
                           'header',
                           'footer',
                           'start_at',
                           'end_at',
                           'use_yn',
                           'deleted',
                           'created',
                           'updated',
                           'buildings',
                           'categories']}

exhibition_product = {'SQL' :"select * " + (
                            " from tb_exhibition_product"),
                                         
              'columns' : ['id','exhibition','product','seq','created','product_type']}

In [123]:
curs.execute(order['SQL'])

dataframe_order = pd.DataFrame(curs)

dataframe_order.columns = order['columns']

In [11]:
curs.execute(fee['SQL'])

dataframe_fee = pd.DataFrame(curs)

dataframe_fee.columns = fee['columns']

In [80]:
curs.execute(order_prod['SQL'])

dataframe_order_prod = pd.DataFrame(curs)

dataframe_order_prod.columns = order_prod['columns']

In [149]:
curs.execute(prod_fee['SQL'])

dataframe_prod_fee = pd.DataFrame(curs)

dataframe_prod_fee.columns = prod_fee['columns']

In [14]:
curs.execute(refund_order['SQL'])

dataframe_refund_order = pd.DataFrame(curs)

dataframe_refund_order.columns = refund_order['columns']

In [15]:
curs.execute(exhibition['SQL'])

dataframe_exhibit = pd.DataFrame(curs)

dataframe_exhibit.columns = exhibition['columns']

In [16]:
curs.execute(exhibition_product['SQL'])

dataframe_exhibit_prod = pd.DataFrame(curs)

dataframe_exhibit_prod.columns =  exhibition_product['columns']

In [17]:
def type_extractor(x, types, recursive = None):
    try :
        if recursive is None:
            return x[0][types]
        else:
            return type_extractor([x[0][types]], 
                       types = recursive)
    except:
        return 'NaN'

In [18]:
def making_product_dictionary(x):
    orderProductId = x['orderProductId'].to_list()
    productId = x['product']
    #productId.index = productId.apply(lambda x : x['id'])
    productId = productId.to_list()
    brand = x['brand'].apply(lambda x : x['id']).to_list()
    return DataFrame({'orderProductId' : [orderProductId],
                       'productId' : [productId],
                       'brandId' : [brand]})

# 1. DataEDA

## 1) dataframe_exhibit

### (1) NULL checking

In [19]:
dataframe_exhibit.apply(lambda x : x.isna().value_counts())

id  name  path  description  header  footer  start_at  end_at  \
False  72.0  72.0  72.0         72.0    72.0     NaN       NaN     NaN   
True    NaN   NaN   NaN          NaN     NaN    72.0      72.0    72.0   

       use_yn  deleted  created  updated  buildings  categories  
False    72.0     72.0     72.0     72.0         28          47  
True      NaN      NaN      NaN      NaN         44          25

In [20]:
dataframe_exhibit[['id','name','path','description','created','updated']]

id            name         path                    description  \
0   68        test1111     test1111                         테스트111   
1   90      BlackLabel   BlackLabel       블랙라벨 큐레이션1 : 여성 - 데일리/심플   
2    2  singapore_test  Black_Label         싱가폴 큐레이션 test (211020)   
3   26           style        style                     5월 스타일 기획전   
4    7        뭐야 도대체??     뭐야 도대체??                       뭐야 도대체??   
..  ..             ...          ...                            ...   
67  64            YOCO         YOCO  대만 업체 엔젤스 도어/ YOCO Collection   
68   5           kids2        kids2                       테스트_kids   
69  37     summer_sale  summer_sale          21년 7월 [여름 땡! SALE대전]   
70  65         krstyle      krstyle                       대만 쇼핑물 7   
71  54          LOVFEE       LOVFEE                        대만 쇼핑물2   

                            created                          updated  
0  2021-12-13 07:10:58.672236+00:00 2021-12-13 07:10:58.672236+00:00  
1  2022-01-11 09:24:06.145000+00:00 2022-02-11 05:39:36.766000+00:00  
2  2020-08-20 06:49:26.921861+00:00 2022-04-18 08:59:23.770061+00:00  
3  2021-05-21 05:52:36.007230+00:00 2021-05-28 01:11:06.855146+00:00  
4  2021-01-25 09:36:42.217455+00:00 2021-01-27 05:22:48.310576+00:00  
..                              ...                              ...  
67 2021-11-12 02:56:04.665302+00:00 2021-11-12 08:03:48.214708+00:00  
68 2021-01-25 08:47:15.050454+00:00 2021-01-27 05:22:39.838382+00:00  
69 2021-07-22 10:51:14.934287+00:00 2021-08-03 06:36:46.622898+00:00  
70 2021-11-15 09:07:53.548939+00:00 2021-11-15 09:08:36.547651+00:00  
71 2021-10-31 13:17:22.410359+00:00 2021-10-31 13:20:16.394682+00:00  

[72 rows x 6 columns]

## 2) dataframe_exhibit_prod

### (1) created unique checking

In [21]:
dataframe_exhibit_prod.groupby('exhibition')['created'].unique()

exhibition
1                    [2022-06-16 00:59:15.346349+00:00]
2     [2022-01-27 02:03:12.798935+00:00, 2022-01-27 ...
3     [2020-10-30 06:14:23.253071+00:00, 2020-10-30 ...
4                    [2022-06-09 05:57:57.972873+00:00]
5     [2021-01-25 09:14:53.813559+00:00, 2021-01-25 ...
                            ...                        
66    [2021-11-17 09:22:26.601010+00:00, 2021-11-17 ...
67    [2021-11-18 10:03:06.193176+00:00, 2021-11-18 ...
68    [2022-01-27 01:58:20.062127+00:00, 2022-01-27 ...
71                   [2022-06-08 05:05:14.341374+00:00]
72                   [2022-06-09 05:56:57.717577+00:00]
Name: created, Length: 64, dtype: object

# 2. dataPreprocessing

### (1) dm_orderseries

In [19]:
def type_extractor(x, types, recursive = None):
        try :
            if recursive is None:
                return x[0][types]
            else:
                return type_extractor([x[0][types]], 
                               types = recursive)
        except:
            return 'NaN'

In [20]:
def list_merge_by_columns(x, columns):
    result = x[columns].apply(lambda x : [x.to_list()], axis = 0)
    return result

In [21]:
def list_merge_various_exhibit(x, columns = ['name',
                                               'path',
                                               'description',
                                               'exhibition',
                                               'created_exhibit_prod',
                                               'exhibition_prod_id',
                                               'created_exhibit',
                                              'terminated_date',
                                              'due_terminated',
                                              'is_exhibition_included']):
    
    result_frame = x[x['orderProductId'].duplicated(keep = False)].groupby('orderProductId').apply(
        lambda x : list_merge_by_columns(x, columns))
    
    result_frame.index.name = 'orderProductId'
    result_frame.columns = columns
    result_frame = result_frame.reset_index()

    result_frame = pd.merge(x.drop(columns, axis = 1).drop_duplicates('orderProductId', keep = 'first'),
                             result_frame,
                            how = 'right',
                            on = 'orderProductId')
    x = x.drop_duplicates('orderProductId', keep = False).reset_index(drop = True)

    result_frame = pd.concat([result_frame, x], axis = 0).reset_index(drop = True)
    return result_frame

In [22]:
dm_orderseries = dataframe_order_prod.copy()

dm_orderseries['product'] = dm_orderseries['product'].apply(lambda x : type_extractor([x], 'id'))

dm_orderseries['brand'] = dm_orderseries['brand'].apply(lambda x : type_extractor([x], 'id'))

dm_orderseries =  dm_orderseries.rename({'product' : 'productId',
                                         'brand' : 'brandId'}, axis = 1)

In [153]:
order_prod_fee = pd.merge(dataframe_order_prod,
                             dataframe_prod_fee,
                             on = ['orderProductId', 'orderId'])

### (2) exhibit

In [30]:
dataframe_exhibit = dataframe_exhibit.rename({'id' : 'exhibition',
                                             'updated' : 'terminated_date'}, axis = 1)
dataframe_exhibit_prod = dataframe_exhibit_prod.rename({'id' : 'exhibition_prod_id',
                                                        'product' : 'productId'}, axis = 1)

In [31]:
exhibit_prod_merge = pd.merge(dataframe_exhibit,
                                 dataframe_exhibit_prod,
                                on = 'exhibition',
                                suffixes = ('_exhibit', '_exhibit_prod'))

In [32]:
exhibit_prod_merge.loc[exhibit_prod_merge['use_yn'] == 'Y', 'terminated_date'] = pd._libs.NaT

In [34]:
exhibit_prod_merge = exhibit_prod_merge[['exhibition', 
                                        'name', 
                                        'path', 
                                        'description', 
                                        'use_yn', 
                                        'deleted',
                                        'created_exhibit',
                                         'terminated_date',
                                        'exhibition_prod_id',
                                        'productId',
                                        'created_exhibit_prod']]

In [36]:
temp_frame = pd.merge(dm_orderseries, exhibit_prod_merge, how = 'left', on = 'productId')
temp_frame['due_terminated'] = (temp_frame['createdAt'] < temp_frame['terminated_date'])
temp_frame.loc[temp_frame['use_yn'] == 'Y', 'due_terminated'] = True
temp_frame['is_exhibition_included'] = (temp_frame['createdAt'] > temp_frame['created_exhibit_prod']) & (
                temp_frame['due_terminated'])

In [37]:
temp_frame = list_merge_various_exhibit(temp_frame)

In [38]:
dm_orderseries = pd.merge(dm_orderseries,
                                    temp_frame[['orderProductId',
                                                'name',
                                                'path',
                                                'description',
                                                'exhibition',
                                               'is_exhibition_included']],
                                     how = 'left',
                                     on = 'orderProductId')

In [39]:
dm_orderseries

orderProductId  orderId productId  buyerId  brandId  finalQuantity  \
0             14368304  1007866   5860440   103700    15697            NaN   
1             14368306  1007867   5182168   110995    14706            NaN   
2             14368305  1007867   5375043   110995    14706            NaN   
3             14368312  1007868   5813960    62821    17636            NaN   
4             14368311  1007868   5813960    62821    17636            NaN   
...                ...      ...       ...      ...      ...            ...   
242066        14611344  1030885   5455829    63864    13518            NaN   
242067        14611343  1030885   5455829    63864    13518            NaN   
242068        14611342  1030885   5455829    63864    13518            NaN   
242069        14611341  1030885   5455840    63864    13518            NaN   
242070        14611340  1030885   5847877    63864    12888            NaN   

       finalKRW  unclepickup  inspection_quantity  \
0          None          NaN                  NaN   
1          None          NaN                  NaN   
2          None          NaN                  NaN   
3          None          NaN                  NaN   
4          None          NaN                  NaN   
...         ...          ...                  ...   
242066     None          NaN                  NaN   
242067     None          NaN                  NaN   
242068     None          NaN                  NaN   
242069     None          NaN                  NaN   
242070     None          NaN                  NaN   

                              createdAt name path description exhibition  \
0      2021-05-01 00:17:41.062428+00:00  NaN  NaN         NaN        NaN   
1      2021-05-01 00:21:28.484363+00:00  NaN  NaN         NaN        NaN   
2      2021-05-01 00:21:28.484363+00:00  NaN  NaN         NaN        NaN   
3      2021-05-01 00:26:15.704981+00:00  NaN  NaN         NaN        NaN   
4      2021-05-01 00:26:15.704981+00:00  NaN  NaN         NaN        NaN   
...                                 ...  ...  ...         ...        ...   
242066 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
242067 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
242068 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
242069 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
242070 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   

       is_exhibition_included  
0                       False  
1                       False  
2                       False  
3                       False  
4                       False  
...                       ...  
242066                  False  
242067                  False  
242068                  False  
242069                  False  
242070                  False  

[242071 rows x 15 columns]

In [40]:
dataframe_order_prod

orderProductId  orderId  \
0             14368304  1007866   
1             14368306  1007867   
2             14368305  1007867   
3             14368312  1007868   
4             14368311  1007868   
...                ...      ...   
242066        14611344  1030885   
242067        14611343  1030885   
242068        14611342  1030885   
242069        14611341  1030885   
242070        14611340  1030885   

                                                  product  buyerId  \
0       {'id': 5860440, 'CNY': 89.29, 'KRW': 15000.0, ...   103700   
1       {'id': 5182168, 'CNY': 17.86, 'KRW': 3000.0, '...   110995   
2       {'id': 5375043, 'CNY': 29.76, 'KRW': 5000.0, '...   110995   
3       {'id': 5813960, 'CNY': 142.86, 'KRW': 24000.0,...    62821   
4       {'id': 5813960, 'CNY': 142.86, 'KRW': 24000.0,...    62821   
...                                                   ...      ...   
242066  {'id': 5455829, 'CNY': 59.52, 'KRW': 10000.0, ...    63864   
242067  {'id': 5455829, 'CNY': 59.52, 'KRW': 10000.0, ...    63864   
242068  {'id': 5455829, 'CNY': 59.52, 'KRW': 10000.0, ...    63864   
242069  {'id': 5455840, 'CNY': 77.38, 'KRW': 13000.0, ...    63864   
242070  {'id': 5847877, 'CNY': 130.95, 'KRW': 22000.0,...    63864   

                                                    brand  finalQuantity  \
0       {'id': 15697, 'data': {'tel': '01057945978', '...            NaN   
1       {'id': 14706, 'data': {'tel': '0263666537', 'b...            NaN   
2       {'id': 14706, 'data': {'tel': '0263666537', 'b...            NaN   
3       {'id': 17636, 'data': {'tel': '', 'bank': {'na...            NaN   
4       {'id': 17636, 'data': {'tel': '', 'bank': {'na...            NaN   
...                                                   ...            ...   
242066  {'id': 13518, 'data': {'tel': '0222360903', 'b...            NaN   
242067  {'id': 13518, 'data': {'tel': '0222360903', 'b...            NaN   
242068  {'id': 13518, 'data': {'tel': '0222360903', 'b...            NaN   
242069  {'id': 13518, 'data': {'tel': '0222360903', 'b...            NaN   
242070  {'id': 12888, 'data': {'tel': '01032926857', '...            NaN   

       finalKRW  unclepickup  inspection_quantity  \
0          None          NaN                  NaN   
1          None          NaN                  NaN   
2          None          NaN                  NaN   
3          None          NaN                  NaN   
4          None          NaN                  NaN   
...         ...          ...                  ...   
242066     None          NaN                  NaN   
242067     None          NaN                  NaN   
242068     None          NaN                  NaN   
242069     None          NaN                  NaN   
242070     None          NaN                  NaN   

                              createdAt  
0      2021-05-01 00:17:41.062428+00:00  
1      2021-05-01 00:21:28.484363+00:00  
2      2021-05-01 00:21:28.484363+00:00  
3      2021-05-01 00:26:15.704981+00:00  
4      2021-05-01 00:26:15.704981+00:00  
...                                 ...  
242066 2021-05-31 23:59:27.649125+00:00  
242067 2021-05-31 23:59:27.649125+00:00  
242068 2021-05-31 23:59:27.649125+00:00  
242069 2021-05-31 23:59:27.649125+00:00  
242070 2021-05-31 23:59:27.649125+00:00  

[242071 rows x 10 columns]

### (3) final ~

In [154]:
order_prod_fee_final = order_prod_fee.copy()

In [155]:
order_prod_fee_final['product'] = order_prod_fee_final['product'].apply(lambda x : type_extractor([x], 'id'))
order_prod_fee_final['brand'] = order_prod_fee_final['brand'].apply(lambda x : type_extractor([x], 'id'))
order_prod_fee_final = order_prod_fee_final.rename({'product' : 'productId',
                                        'brand' : 'brandId'}, axis = 1)

In [156]:
order_prod_fee_final['unit_quality_inspection_fee_amount'] = order_prod_fee_final['unit_quality_inspection_fee_amount'].astype('float')
order_prod_fee_final['unit_handling_fee_amount'] = order_prod_fee_final['unit_handling_fee_amount'].astype('float')
order_prod_fee_final['unit_product_supply'] = order_prod_fee_final['unit_product_supply'].astype('float')
order_prod_fee_final['finalKRW'] = order_prod_fee_final['finalKRW'].astype('float')
order_prod_fee_final['unit_product_vat'] = order_prod_fee_final['unit_product_vat'].astype('float')
#order_prod_fee['finalQuantity'] = order_prod_fee['finalQuantity'].astype('float')

In [157]:
order_prod_fee_final['finalQualityInspectionFee'] = order_prod_fee_final['inspection_quantity'] * (
                                                order_prod_fee_final['unit_quality_inspection_fee_amount'])
order_prod_fee_final['finalHandlingFee'] = order_prod_fee_final['unclepickup'] * (
                                                order_prod_fee_final['unit_handling_fee_amount'])
order_prod_fee_final['finalSubtotal'] = order_prod_fee_final['finalQuantity'] * (
                                                order_prod_fee_final['finalKRW'])
order_prod_fee_final['finalTax'] = order_prod_fee_final['finalQuantity'] * (
                                                order_prod_fee_final['unit_product_vat'])

In [158]:
order_prod_fee_final['finalTotalKRW'] = order_prod_fee_final['finalQualityInspectionFee'] + (
                                                order_prod_fee_final['finalHandlingFee']) + (
                                                    order_prod_fee_final['finalSubtotal']) + (
                                                        order_prod_fee_final['finalTax'])

In [159]:
order_prod_fee_final = order_prod_fee_final[['orderProductId',
                                            'orderId',
                                            'buyerId',
                                            'finalQualityInspectionFee',
                                            'finalHandlingFee',
                                            'finalSubtotal',
                                            'finalTax',
                                            'finalTotalKRW',
                                           'createdAt']].dropna()

In [160]:
test = pd.merge(order_prod_fee_final.groupby('orderId')[['finalQualityInspectionFee',
                                            'finalHandlingFee',
                                            'finalSubtotal',
                                            'finalTax',
                                            'finalTotalKRW']].sum().reset_index(),
         dataframe_order,
         on = 'orderId',
         suffixes= ('_unit', ''))

In [161]:
#dataframe_order[dataframe_order['orderId'] == 1007881]

In [162]:
#test[~test['finalTotalKRW'].isna()].T

In [163]:
test = test[np.sort(test.columns)].dropna()

In [164]:
test['finalSubtotalKRW'] = test['finalSubtotalKRW'].astype('float')
test['finalTotalKRW'] = test['finalTotalKRW'].astype('float')
test['finalTotalKRW_unit'] = test['finalTotalKRW_unit'].astype('float')
test['finalHandlingFee'] = test['finalHandlingFee'].astype('float') 
test['finalHandlingFee_unit'] = test['finalHandlingFee_unit'].astype('float') 
test['finalShippingCost'] = test['finalShippingCost'].astype('float')

In [165]:
test['finalTotalKRW_unit'] = test['finalTotalKRW_unit'] + test['finalShippingCost']

In [166]:
plt.figure(figsize = (20,10))
(test['finalSubtotalKRW'] - test['finalSubtotal']).hist(bins = 1000)

<AxesSubplot:>

In [167]:
(test['finalSubtotalKRW'] - test['finalSubtotal']).unique()

array([     0., -40000.])

In [168]:
test[(test['finalSubtotalKRW'] - test['finalSubtotal']) != 0]

buyerId                        createdAt  finalHandlingFee  \
1825   130052 2021-05-12 09:34:46.140797+00:00          103540.0   

      finalHandlingFee_unit  finalQualityInspectionFee  finalShippingCost  \
1825               102490.0                   142200.0            16500.0   

      finalSubtotal  finalSubtotalKRW   finalTax  finalTax_unit  \
1825      2883000.0         2843000.0  284300.00       288300.0   

      finalTotalKRW  finalTotalKRW_unit handlingFeeKRW  orderId subtotalKRW  \
1825      3384540.0           3432490.0      105700.00  1015644  2901000.00   

         taxKRW  
1825  290100.00

- 단 한건을 제외하고, finalQuantity * finalKRW = finalSubtotalKRW이다.

In [169]:
test[(test['finalTotalKRW'] - test['finalTotalKRW_unit']) != 0]

buyerId                        createdAt  finalHandlingFee  \
0       94841 2021-05-01 00:29:53.015945+00:00           15390.0   
1      117131 2021-05-01 00:46:27.788850+00:00            1530.0   
25      86684 2021-05-01 05:37:21.992063+00:00            1620.0   
33     136529 2021-05-01 08:04:30.677697+00:00            3330.0   
34      66198 2021-05-01 08:17:32.368392+00:00            4320.0   
...       ...                              ...               ...   
5462    68451 2021-05-31 16:36:14.134551+00:00            5200.0   
5464   113083 2021-05-31 17:12:24.722948+00:00            1630.0   
5465   120162 2021-05-31 17:35:39.608353+00:00            1460.0   
5469    83652 2021-05-31 18:35:04.455790+00:00           33960.0   
5470   137282 2021-05-31 18:49:55.717940+00:00            7410.0   

      finalHandlingFee_unit  finalQualityInspectionFee  finalShippingCost  \
0                   17500.0                    12700.0            26400.0   
1                    1930.0                     2500.0             9900.0   
25                   1370.0                     2000.0             9900.0   
33                   4490.0                     3900.0             9900.0   
34                   5590.0                     2100.0             9900.0   
...                     ...                        ...                ...   
5462                 4210.0                     5200.0             6600.0   
5464                  940.0                     1500.0             6600.0   
5465                 1860.0                     2900.0             6600.0   
5469                35100.0                    60300.0             6600.0   
5470                 6180.0                     5200.0             9900.0   

      finalSubtotal  finalSubtotalKRW  finalTax  finalTax_unit  finalTotalKRW  \
0          424000.0          424000.0  42400.00        42400.0       520890.0   
1           42000.0           42000.0   4200.00         4200.0        60130.0   
25          45000.0           45000.0   4500.00         4500.0        63020.0   
33          92000.0           92000.0   9200.00         9200.0       118330.0   
34         119000.0          119000.0  11900.00        11900.0       147220.0   
...             ...               ...       ...            ...            ...   
5462       143000.0          143000.0  14300.00        14300.0       174300.0   
5464        45000.0           45000.0   4500.00         4500.0        59230.0   
5465        40000.0           40000.0   4000.00         4000.0        54960.0   
5469       937000.0          937000.0  93700.00        93700.0      1131560.0   
5470       204000.0          204000.0  20400.00        20400.0       246910.0   

      finalTotalKRW_unit handlingFeeKRW  orderId subtotalKRW     taxKRW  
0               523000.0       24220.00  1007869   667000.00   66700.00  
1                60530.0        2220.00  1007881    61000.00    6100.00  
25               62770.0        1620.00  1007986    45000.00    4500.00  
33              119490.0        3330.00  1008038    92000.00    9200.00  
34              148490.0        4320.00  1008040   119000.00   11900.00  
...                  ...            ...      ...         ...        ...  
5462            173310.0        5600.00  1030796   154000.00   15400.00  
5464             58540.0        1630.00  1030809    45000.00    4500.00  
5465             55360.0        2220.00  1030816    61000.00    6100.00  
5469           1132700.0       36760.00  1030833  1014000.00  101400.00  
5470            245680.0        7410.00  1030834   204000.00   20400.00  

[1147 rows x 16 columns]

In [74]:
test[(test['finalHandlingFee'] - test['finalHandlingFee_unit']) != 0].T

3      \
buyerId                                               94841   
createdAt                  2021-05-01 00:29:53.015945+00:00   
finalHandlingFee                                    15390.0   
finalHandlingFee_unit                               17500.0   
finalQualityInspectionFee                           12700.0   
finalShippingCost                                   26400.0   
finalSubtotal                                      424000.0   
finalSubtotalKRW                                   424000.0   
finalTax                                           42400.00   
finalTax_unit                                       42400.0   
finalTotalKRW                                      520890.0   
finalTotalKRW_unit                                 523000.0   
orderId                                             1007869   

                                                      15     \
buyerId                                              117131   
createdAt                  2021-05-01 00:46:27.788850+00:00   
finalHandlingFee                                     1530.0   
finalHandlingFee_unit                                1930.0   
finalQualityInspectionFee                            2500.0   
finalShippingCost                                    9900.0   
finalSubtotal                                       42000.0   
finalSubtotalKRW                                    42000.0   
finalTax                                            4200.00   
finalTax_unit                                        4200.0   
finalTotalKRW                                       60130.0   
finalTotalKRW_unit                                  60530.0   
orderId                                             1007881   

                                                      120    \
buyerId                                               86684   
createdAt                  2021-05-01 05:37:21.992063+00:00   
finalHandlingFee                                     1620.0   
finalHandlingFee_unit                                1370.0   
finalQualityInspectionFee                            2000.0   
finalShippingCost                                    9900.0   
finalSubtotal                                       45000.0   
finalSubtotalKRW                                    45000.0   
finalTax                                            4500.00   
finalTax_unit                                        4500.0   
finalTotalKRW                                       63020.0   
finalTotalKRW_unit                                  62770.0   
orderId                                             1007986   

                                                      172    \
buyerId                                              136529   
createdAt                  2021-05-01 08:04:30.677697+00:00   
finalHandlingFee                                     3330.0   
finalHandlingFee_unit                                4490.0   
finalQualityInspectionFee                            3900.0   
finalShippingCost                                    9900.0   
finalSubtotal                                       92000.0   
finalSubtotalKRW                                    92000.0   
finalTax                                            9200.00   
finalTax_unit                                        9200.0   
finalTotalKRW                                      118330.0   
finalTotalKRW_unit                                 119490.0   
orderId                                             1008038   

                                                      174    \
buyerId                                               66198   
createdAt                  2021-05-01 08:17:32.368392+00:00   
finalHandlingFee                                     4320.0   
finalHandlingFee_unit                                5590.0   
finalQualityInspectionFee                            2100.0   
finalShippingCost                                    9900.0   
finalSubtotal                                      119000.0   
finalSubtotalKRW  

In [ ]:
pd.merge(dm_orderseries,
         order_prod_fee,
         on = ['orderProductId','orderId','productId','buyerId','brandId'])

### (4) init

In [177]:
order_prod_fee_init = order_prod_fee.copy()

In [178]:
order_prod_fee_init['product'] = order_prod_fee_init['product'].apply(lambda x : type_extractor([x], 'id'))
order_prod_fee_init['brand'] = order_prod_fee_init['brand'].apply(lambda x : type_extractor([x], 'id'))
order_prod_fee_init = order_prod_fee_init.rename({'product' : 'productId',
                                        'brand' : 'brandId'}, axis = 1)

In [179]:
order_prod_fee_init['initHandlingFee'] = order_prod_fee_init['quantity'] * (
                                         order_prod_fee_init['unit_handling_fee_amount'])
order_prod_fee_init['initQualityInspectionFee'] = order_prod_fee_init['quantity'] * (
                                                  order_prod_fee_init['unit_quality_inspection_fee_amount'])
order_prod_fee_init['initSubtotal'] = order_prod_fee_init['quantity'] * (
                                      order_prod_fee_init['unit_product_supply'])
order_prod_fee_init['initTax'] = order_prod_fee_init['quantity'] * (
                                 order_prod_fee_init['unit_product_vat'])

In [180]:
order_prod_fee_init = order_prod_fee_init[['orderProductId',
                                            'orderId',
                                            'productId',
                                            'buyerId',
                                            'brandId',
                                            'createdAt',
                                            'initHandlingFee',
                                            'initQualityInspectionFee',
                                            'initSubtotal',
                                            'initTax']]

In [181]:
test = pd.merge(order_prod_fee_init.groupby('orderId')[['initHandlingFee', 
                                                   'initQualityInspectionFee',
                                                   'initSubtotal', 
                                                   'initTax']].sum().reset_index(),
         dataframe_order,
         on = 'orderId',
         suffixes= ('_unit', ''))

In [182]:
test

orderId initHandlingFee initQualityInspectionFee initSubtotal  \
0      1007866          540.00                   500.00     15000.00   
1      1007867          290.00                   800.00      8000.00   
2      1007868         3630.00                  3500.00    100000.00   
3      1007869        24220.00                 16500.00    667000.00   
4      1007870         2490.00                  1500.00     69000.00   
...        ...             ...                      ...          ...   
22039  1030881         1660.00                  1000.00     46000.00   
22040  1030882         1640.00                  1000.00     45000.00   
22041  1030883          940.00                  1000.00     26000.00   
22042  1030884          940.00                  1000.00     26000.00   
22043  1030885         2350.00                  2700.00     65000.00   

        initTax  buyerId finalTotalKRW finalSubtotalKRW finalHandlingFee  \
0       1500.00   103700          None             None             None   
1        800.00   110995          None             None             None   
2      10000.00    62821          None             None             None   
3      66700.00    94841     520890.00        424000.00         15390.00   
4       6900.00   110995          None             None             None   
...         ...      ...           ...              ...              ...   
22039   4600.00    27178      59860.00         46000.00          1660.00   
22040   4500.00   137151          0.00             0.00             0.00   
22041   2600.00   135505          None             None             None   
22042   2600.00   135505      33840.00         26000.00           940.00   
22043   6500.00    63864          None             None             None   

      finalShippingCost  finalTax handlingFeeKRW subtotalKRW    taxKRW  \
0                  None      None         540.00    15000.00   1500.00   
1                  None      None         290.00     8000.00    800.00   
2                  None      None        3630.00   100000.00  10000.00   
3              26400.00  42400.00       24220.00   667000.00  66700.00   
4                  None      None        2490.00    69000.00   6900.00   
...                 ...       ...            ...         ...       ...   
22039           6600.00   4600.00        1660.00    46000.00   4600.00   
22040              None      0.00        1640.00    45000.00   4500.00   
22041              None      None         940.00    26000.00   2600.00   
22042           3300.00   2600.00         940.00    26000.00   2600.00   
22043              None      None        2350.00    65000.00   6500.00   

                             createdAt  
0     2021-05-01 00:17:41.062428+00:00  
1     2021-05-01 00:21:28.484363+00:00  
2     2021-05-01 00:26:15.704981+00:00  
3     2021-05-01 00:29:53.015945+00:00  
4     2021-05-01 00:31:38.054822+00:00  
...                                ...  
22039 2021-05-31 23:50:47.365443+00:00  
22040 2021-05-31 23:52:46.168460+00:00  
22041 2021-05-31 23:54:30.931691+00:00  
22042 2021-05-31 23:56:10.716838+00:00  
22043 2021-05-31 23:59:27.649125+00:00  

[22044 rows x 15 columns]

In [183]:
(test['handlingFeeKRW'] - test['initHandlingFee']).unique()

array([Decimal('0.00'), Decimal('2080.00')], dtype=object)

In [184]:
(test['subtotalKRW'] - test['initSubtotal']).unique()

array([Decimal('0.00'), Decimal('26000.00')], dtype=object)

In [185]:
(test['initTax'] - test['taxKRW']).unique()

array([Decimal('0.00')], dtype=object)

In [186]:
test[(test['handlingFeeKRW'] - test['initHandlingFee']) != 0]

orderId initHandlingFee initQualityInspectionFee initSubtotal initTax  \
13752  1022136       782720.00                587040.00   9784000.00    0.00   

       buyerId finalTotalKRW finalSubtotalKRW finalHandlingFee  \
13752    42191   11609660.00       9749000.00        779920.00   

      finalShippingCost finalTax handlingFeeKRW subtotalKRW taxKRW  \
13752         495800.00     0.00      784800.00  9810000.00   0.00   

                             createdAt  
13752 2021-05-20 07:14:46.515120+00:00

In [187]:
test[(test['subtotalKRW'] - test['initSubtotal']) != 0]

orderId initHandlingFee initQualityInspectionFee initSubtotal initTax  \
13752  1022136       782720.00                587040.00   9784000.00    0.00   

       buyerId finalTotalKRW finalSubtotalKRW finalHandlingFee  \
13752    42191   11609660.00       9749000.00        779920.00   

      finalShippingCost finalTax handlingFeeKRW subtotalKRW taxKRW  \
13752         495800.00     0.00      784800.00  9810000.00   0.00   

                             createdAt  
13752 2021-05-20 07:14:46.515120+00:00

- 단 한건을 제외하고

1) handlingFeeKRW = quantity * unit_handling_fee_amount이다.

2) subtotalKRW = quantity * unit_product_supply이다.

3) taxKRW = quantity * unit_product_vat이다.

In [189]:
pd.merge(dm_orderseries,
         order_prod_fee_init,
         on = ['orderProductId','orderId','productId','buyerId','brandId'])

orderProductId  orderId productId  buyerId  brandId  finalQuantity  \
0             14368304  1007866   5860440   103700    15697            NaN   
1             14368306  1007867   5182168   110995    14706            NaN   
2             14368305  1007867   5375043   110995    14706            NaN   
3             14368312  1007868   5813960    62821    17636            NaN   
4             14368311  1007868   5813960    62821    17636            NaN   
...                ...      ...       ...      ...      ...            ...   
197838        14611344  1030885   5455829    63864    13518            NaN   
197839        14611343  1030885   5455829    63864    13518            NaN   
197840        14611342  1030885   5455829    63864    13518            NaN   
197841        14611341  1030885   5455840    63864    13518            NaN   
197842        14611340  1030885   5847877    63864    12888            NaN   

       finalKRW  unclepickup  inspection_quantity  \
0          None          NaN                  NaN   
1          None          NaN                  NaN   
2          None          NaN                  NaN   
3          None          NaN                  NaN   
4          None          NaN                  NaN   
...         ...          ...                  ...   
197838     None          NaN                  NaN   
197839     None          NaN                  NaN   
197840     None          NaN                  NaN   
197841     None          NaN                  NaN   
197842     None          NaN                  NaN   

                            createdAt_x name path description exhibition  \
0      2021-05-01 00:17:41.062428+00:00  NaN  NaN         NaN        NaN   
1      2021-05-01 00:21:28.484363+00:00  NaN  NaN         NaN        NaN   
2      2021-05-01 00:21:28.484363+00:00  NaN  NaN         NaN        NaN   
3      2021-05-01 00:26:15.704981+00:00  NaN  NaN         NaN        NaN   
4      2021-05-01 00:26:15.704981+00:00  NaN  NaN         NaN        NaN   
...                                 ...  ...  ...         ...        ...   
197838 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
197839 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
197840 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
197841 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   
197842 2021-05-31 23:59:27.649125+00:00  NaN  NaN         NaN        NaN   

       is_exhibition_included                      createdAt_y  \
0                       False 2021-05-01 00:17:41.062428+00:00   
1                       False 2021-05-01 00:21:28.484363+00:00   
2                       False 2021-05-01 00:21:28.484363+00:00   
3                       False 2021-05-01 00:26:15.704981+00:00   
4                       False 2021-05-01 00:26:15.704981+00:00   
...                       ...                              ...   
197838                  False 2021-05-31 23:59:27.649125+00:00   
197839                  False 2021-05-31 23:59:27.649125+00:00   
197840                  False 2021-05-31 23:59:27.649125+00:00   
197841                  False 2021-05-31 23:59:27.649125+00:00   
197842                  False 2021-05-31 23:59:27.649125+00:00   

       initHandlingFee initQualityInspectionFee initSubtotal  initTax  
0               540.00                   500.00     15000.00  1500.00  
1               110.00                   300.00      3000.00   300.00  
2               180.00                   500.00      5000.00   500.00  
3               870.00                   500.00     24000.00  2400.00  
4               870.00                   500.00     24000.00  2400.00  
...                ...                      ...          ...      ...  
197838          360.00                   500.00     10000.00  1000.00  
197839          360.00                   500.00     10000.00  1000.00  
197840          360.00                   500.00     10000.00  1000.00  
197841  

### (5) dc

In [190]:
order_prod_fee_dc = order_prod_fee.copy()